In [6]:
using DynamicPolynomials, LinearAlgebra, SparseArrays

println("***Problem setting***")

n=1000

println("Number of variable: n=",n)
println("====================")

@ncpolyvar x[1:n]# variables
function star_algebra(mom)
    if mom==1
        return mom
    else
        ind=mom.z .>0
        vars=mom.vars[ind]
        exp=mom.z[ind]
        return prod(vars[i]^exp[i] for i in length(exp):-1:1)
    end
end
function generate_random_poly(T::UnitRange{Int64})
    v=reverse(monomials(x[T],0:2))
    v+=star_algebra.(v)
    v=v./2
    c=2*rand(Float64,length(v)).-1
    return c'*v
end

# unit sphere constraint
u=10
p=floor(Int64,n/u)

I=Vector{UnitRange{Int64}}(undef,p)
I[1]=1:u
I[2:p-1]=[u*(j-1):u*j for j in 2:p-1]
I[p]=u*(p-1):n

# random quadratic objective function f
f=sum(generate_random_poly(I[j]) for j in 1:p)

f=f/100
g=-(x.^2).+1/u



m=length(g)
println("Number of inequality constraints: m=",m)
println("====================")

l=ceil(Int64, n/7)

r=floor(Int64,l/p)
W=[(j-1)*r+1:j*r for j in 1:p-1]
append!(W,[(p-1)*r+1:l])

h=Vector{Polynomial{false,Float64}}(undef,l)


randx=2*rand(n).-1
randx=randx./sqrt(u)

for j in 1:p
    for i in W[j]
        h[i]=generate_random_poly(I[j])
        h[i]-=h[i](x => randx) #make constraints feasible
    end
end

l=length(h)
println("Number of equality constraints: l=",l)
println("====================")


***Problem setting***
Number of variable: n=1000
Number of inequality constraints: m=1000
Number of equality constraints: l=143


In [7]:
include("../src/ctpNCPOP.jl")
using .ctpNCPOP

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=ctpNCPOP.get_info(x,f,g,h);

In [11]:
k=2

println("Relaxed order: k=",k)
println("====================")

Relaxed order: k=2


In [12]:
include("../src/ctpNCPOP.jl")
using .ctpNCPOP

opt_val1=ctpNCPOP.POP_CS_CGAL(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,
    maxit=Int64(1e10),tol=1e-3,use_eqcons_to_get_constant_trace=false,check_tol_each_iter=true)

------------------------------------------------------
The clique sizes of varibles:
[11, 10]
[99, 1]
------------------------------------------------------
  Number of cliques: p=100
  Largest clique size: u=11


  Largest constant trace: a=3.3100000000000014
  Number of blocks: omega=1199
  Number of equality consraints: zeta=169664
  Size of the largest block: s^max=133
Modeling time:
 71.114504 seconds (532.02 M allocations: 81.260 GiB, 14.65% gc time)
iter=1   val=-7.615456750600549   gap=4.492128978875036e-7   feas=35.984695714706874
iter=2   val=-5.839775777883782   gap=0.01303595364623216   feas=22.215072503040066
iter=4   val=-4.12575589809567   gap=0.15052221827220783   feas=9.791565540385811
iter=8   val=-3.723703085571032   gap=0.013140085310203343   feas=7.379701207152166
iter=16   val=-3.2223035296519167   gap=0.03712463834727638   feas=3.615674181812576
iter=32   val=-2.980555220479911   gap=0.009632473069823838   feas=2.1363472062857856
iter=64   val=-2.762287089526606   gap=0.007558981360055946   feas=1.1934010440881027
iter=128   val=-2.577578758300037   gap=0.002534565700145187   feas=0.6200992998095567
iter=256   val=-2.4728034913051906   gap=0.005940097331950341   feas=0.282

-2.4253975619098034

In [10]:
include("../src/ctpNCPOP.jl")
using .ctpNCPOP

opt_val1=ctpNCPOP.POP_CS_CGAL(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,
    maxit=Int64(1e10),tol=5e-3,use_eqcons_to_get_constant_trace=false,check_tol_each_iter=true,check_by_mosek=true)

------------------------------------------------------
The clique sizes of varibles:
[11, 10]
[99, 1]
------------------------------------------------------
  Number of cliques: p=100
  Largest clique size: u=11


  Largest constant trace: a=2.100000000000001
  Number of blocks: omega=1199
  Number of equality consraints: zeta=1540
  Size of the largest block: s^max=12
Mosek:-----------------
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 10428           
  Cones                  : 0               
  Scalar variables       : 8887            
  Matrix variables       : 100             
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 198
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time           

OPTIMAL::TerminationStatusCode = 1